In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}?"
)

prompt = template.format(country_a="Korea", country_b="Malaysia")

chat.predict(prompt)



The distance between Korea and Malaysia is approximately 3,500 kilometers (2,175 miles) when measured in a straight line.

'The distance between Korea and Malaysia is approximately 3,500 kilometers (2,175 miles) when measured in a straight line.'

In [2]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geopraphy expert. You only reply in {language}"),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}? Also, what is your name?")
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_b="Italy",
    country_a="Greece"
)

chat.predict_messages(prompt)

Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ Ελλάδας και Ιταλίας εξαρτάται από την συγκεκριμένη πόλη στην Ελλάδα και την πόλη στην Ιταλία. Γενικά, η απόσταση μεταξύ των δύο χωρών είναι περίπου 800-1000 χιλιόμετρα, ανάλογα με το σημείο αναφοράς.

AIMessageChunk(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ Ελλάδας και Ιταλίας εξαρτάται από την συγκεκριμένη πόλη στην Ελλάδα και την πόλη στην Ιταλία. Γενικά, η απόσταση μεταξύ των δύο χωρών είναι περίπου 800-1000 χιλιόμετρα, ανάλογα με το σημείο αναφοράς.')

In [3]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello, how,are,you")

['Hello', 'how', 'are', 'you']

In [4]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you  are asked will be answered with a comma seperated list of max {max_items}. Do not reply with anything else."),
        ("human", "{question}")
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)


Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Pluto

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [5]:

chain = template | chat | p

chain.invoke({
    "max_items": 5,
    "question": "What are the colors?"
})

Red, blue, green, yellow, orange

['Red', 'blue', 'green', 'yellow', 'orange']

In [6]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
    ]
)

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "Indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic recipe for Chicken Tikka Masala, a popular dish loved by many. Here's a simple recipe for you to try at home:

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tsp ground cumin
- 2 tsp ground coriander
- 1 tsp turmeric
- 1 tsp paprika
- 1 tsp garam masala
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together yogurt, lemon juice, cumin, coriander, turmeric, paprika, garam masala, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can easily substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\n**Vegetarian Tikka Masala**\n\n**Ingredients:**\n- 1 block of firm tofu or paneer, cut into bite-sized cubes\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan option)\n- 2 tbsp lemon juice\n- 2 tsp ground cumin\n- 2 tsp ground coriander\n- 1 tsp turmeric\n- 1 tsp paprika\n- 1 tsp garam masala\n- 1/2 tsp cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 cup coconut cream (or dairy-free heavy cream alternative)\n- Fresh cilantro, chopped (for garnish)\n\n**Instructions:**\n1. Follow the same marinating process as the original recipe, substituting the chicken with tofu or paneer. Coat the tofu or paneer well wit